# Translation with Azure AI Translator

https://learn.microsoft.com/en-us/azure/ai-services/translator/text-translation-overview

<img src="https://github.com/retkowsky/azure-ai-translator/blob/main/webapp1.jpg?raw=true">

In [1]:
import datetime
import gradio as gr
import pandas as pd
import sys

from azure.ai.translation.text import TextTranslationClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.translation.text.models import InputTextItem
from azure.core.exceptions import HttpResponseError
from dotenv import load_dotenv

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 05-Mar-2025 11:14:13


## Azure AI Translator credentials

In [4]:
load_dotenv("azure.env")

azure_ai_translator_key = os.getenv("AZURE_AI_TRANSLATION_KEY")
azure_ai_translator_region = os.getenv("AZURE_AI_TRANSLATION_REGION")

In [5]:
credential = AzureKeyCredential(azure_ai_translator_key)

text_translator = TextTranslationClient(
    credential=credential,
    region=azure_ai_translator_region
)

In [6]:
response = text_translator.get_supported_languages()

In [7]:
def print_languages(label, languages):
        """
        Print supported languages of Azure AI Translator
        """
        print("\033[1;31;34m")
        
        if languages is not None:
            print(f"Number of supported {label} languages = {len(languages)}\n")
            print(f"{label.capitalize()} languages:")

            for idx, (key, value) in enumerate(languages.items(), start=1):
                print(f"{idx:03}\t{key:10} {value.name} ({value.native_name})")

        else:
            print(f"No supported {label} languages.")

In [8]:
print_languages("translation", response.translation)


Number of supported translation languages = 135

Translation languages:
001	af         Afrikaans (Afrikaans)
002	am         Amharic (አማርኛ)
003	ar         Arabic (العربية)
004	as         Assamese (অসমীয়া)
005	az         Azerbaijani (Azərbaycan)
006	ba         Bashkir (Bashkir)
007	bg         Bulgarian (Български)
008	bho        Bhojpuri (भोजपुरी)
009	bn         Bangla (বাংলা)
010	bo         Tibetan (བོད་སྐད་)
011	brx        Bodo (बड़ो)
012	bs         Bosnian (Bosanski)
013	ca         Catalan (Català)
014	cs         Czech (Čeština)
015	cy         Welsh (Cymraeg)
016	da         Danish (Dansk)
017	de         German (Deutsch)
018	doi        Dogri (डोगरी)
019	dsb        Lower Sorbian (Dolnoserbšćina)
020	dv         Divehi (ދިވެހިބަސް)
021	el         Greek (Ελληνικά)
022	en         English (English)
023	es         Spanish (Español)
024	et         Estonian (Eesti)
025	eu         Basque (Euskara)
026	fa         Persian (فارسی)
027	fi         Finnish (Suomi)
028	fil        Filipino (Filipino)


In [9]:
lang_list = []

if response.translation is not None:
    for key, value in response.translation.items():
        lang_list.append(
            {
                "Language_Code": key,
                "Language_Name": value.name,
                "Native_Name": value.native_name,
            }
        )

df_languages = pd.DataFrame(lang_list)
df_languages

,Language_Code,Language_Name,Native_Name
0,af,Afrikaans,Afrikaans
1,am,Amharic,አማርኛ
2,ar,Arabic,العربية
3,as,Assamese,অসমীয়া
4,az,Azerbaijani,Azərbaycan
...,...,...,...
130,yua,Yucatec Maya,Yucatec Maya
131,yue,Cantonese (Traditional),粵語 (繁體)
132,zh-Hans,Chinese Simplified,中文 (简体)
133,zh-Hant,Chinese Traditional,繁體中文 (繁體)


In [10]:
# Get dict from dataframe
language_dict = df_languages.set_index("Language_Code")["Language_Name"].to_dict()
# Full names list
language_full_names = list(language_dict.values())
# Sort
language_full_names.sort()
# Language codes list
language_codes = list(language_dict.keys())
# Sort
language_codes.sort()
# Creation of dict
reverse_language_names = {v: k for k, v in language_dict.items()}

In [11]:
def get_language_code(language_name):
    """
    Function to get language code by language name
    Input: language full name
    Output: language code name
    """
    return reverse_language_names.get(language_name)

In [12]:
print(get_language_code("English"))

en


## Testing

In [13]:
def azure_ai_translator(mytext, source_lang, target_lang):
    """
    Translates text from one language to another using Azure AI Translator.
    
    Args:
        mytext (str): The text to be translated.
        source_lang (str): The full name of the source language.
        target_lang (str): The full name of the target language.
    
    Returns:
        str: The translated text, or None if an error occurs.
    """
    try:
        credential = AzureKeyCredential(azure_ai_translator_key)
        
        text_translator = TextTranslationClient(
            credential=credential, region=azure_ai_translator_region)
        input_text_elements = [mytext]

        # Get language codes
        source_lang_code = get_language_code(source_lang)
        target_lang_code = [get_language_code(target_lang)]
        
        # Response
        response = text_translator.translate(body=input_text_elements,
                                             to_language=target_lang_code)
        translation = response[0] if response else None

        if translation:
            detected_language = translation.detected_language
            if detected_language:
                print(f"Detected languages of the input text: {detected_language.language} with score = {detected_language.score}.")
            for translated_text in translation.translations:
                print(f"\nText to translate to: '{translated_text.to}'")
            return translated_text.text

    except HttpResponseError as exception:
        if exception.error is not None:
            print(f"Error Code: {exception.error.code}")
            print(f"Message: {exception.error.message}")

In [14]:
mytext = "Azure is a cloud computing platform run by Microsoft. It offers access, management, and the development of applications and services through global data centers"

result = azure_ai_translator(mytext, "English", "French")
print(result)

Detected languages of the input text: en with score = 1.0.

Text to translate to: 'fr'
Azure est une plateforme de cloud computing gérée par Microsoft. Elle offre l’accès, la gestion et le développement d’applications et de services par le biais de centres de données mondiaux


In [15]:
mytext = "Quel est le temps de trajet de Paris à Versailles ?"

result = azure_ai_translator(mytext, "French", "Italian")
print(result)

Detected languages of the input text: fr with score = 1.0.

Text to translate to: 'it'
Qual è il tempo di viaggio da Parigi a Versailles?


## Webapp

https://learn.microsoft.com/en-us/azure/ai-services/translator/language-support

In [18]:
image_url = "https://github.com/retkowsky/azure-ai-translator/blob/main/logo.jpg?raw=true"
logo = "<center> <img src= {} width=200px></center>".format(image_url)

translator_webapp = gr.Interface(
    fn=azure_ai_translator,
    inputs=[
        gr.components.Textbox(label="Text to translate"),
        gr.components.Dropdown(label="Source language",
                               choices=language_full_names),
        gr.components.Dropdown(label="Target language",
                               choices=language_full_names),
    ],
    outputs=gr.Text(label="Translated text"),
    cache_examples=False,
    title="Azure AI Translator",
    description=logo,
    #theme="JohnSmith9982/small_and_pretty",
    examples=[
        [
            "Hello. Welcome to this presentation of Azure AI Translator",
            "English",
            "French",
        ],
        [
            "Hola. Bienvenidos a esta presentación de Azure AI Translator",
            "Spanish",
            "Italian",
        ],
        [
            "Bonjour à tous.",
            "French",
            "Arabic",
        ],
    ],
)

translator_webapp.launch(share=True)

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://1e5738d4d8114a12b0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Detected languages of the input text: en with score = 1.0.

Text to translate to: 'fr'
Detected languages of the input text: es with score = 1.0.

Text to translate to: 'it'
Detected languages of the input text: fr with score = 1.0.

Text to translate to: 'ar'
